# Домашнее задание «Продвинутый pandas 2»

In [ ]:
import pandas as pd

# Задание 1

In [ ]:
# Используйте файл с оценками фильмов ml-latest-small/ratings.csv. 
# Посчитайте среднее время жизни пользователей, которые выставили более 100 оценок. 
# Под временем жизни понимается разница между максимальным и минимальным значением столбца 
# timestamp для данного значения userId.

In [ ]:
df = pd.read_csv('ml-latest-small/ratings.csv')

In [ ]:
# Делаем второй датафрейм, где будет подсчитано количество оценок одного пользователя
df_sorted = df.groupby('userId').count()['rating'].reset_index().sort_values('rating', ascending=False)

In [ ]:
# Оставляем только пользователей, у кого оценок больше 100
df_sorted = df_sorted.loc[df_sorted.rating > 100]

In [ ]:
# Создаем датафрейм, где будут только пользователи более 100 оценок
df_sorted = df_sorted.drop(['rating'], axis='columns').head()
joined = df_sorted.merge(df, how='left', on='userId')

In [ ]:
# Создаем функцию для подсчета разницы между максимальным и минимальным значением timestamp
def counting(data):
    return data.timestamp.max() - data.timestamp.min()

In [ ]:
# Создаем объект-группировку по userId из имеющегося датафрейма
obj = joined.groupby('userId')

In [ ]:
# Считаем среднее время жизни пользователей, которые выставили более 100 оценок
mean_time = obj.apply(counting).mean()

# Задание 2

In [ ]:
# Дана статистика услуг перевозок клиентов компании по типам (см. файл с кодом занятия). 
# Необходимо сформировать две таблицы:
# 1. таблицу с тремя типами выручки для каждого client_id без указания адреса клиента
# 2. аналогичную таблицу по типам выручки с указанием адреса клиента
# Обратите внимание, что в процессе объединения таблиц данные не должны теряться.

In [ ]:
rzd = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115],
        'rzd_revenue': [1093, 2810, 10283, 5774, 981]
    }
)
rzd

In [ ]:
auto = pd.DataFrame(
    {
        'client_id': [113, 114, 115, 116, 117],
        'auto_revenue': [57483, 83, 912, 4834, 98]
    }
)

In [ ]:
air = pd.DataFrame(
    {
        'client_id': [115, 116, 117, 118],
        'air_revenue': [81, 4, 13, 173]
    }
)

In [ ]:
client_base = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115, 116, 117, 118],
        'address': ['Комсомольская 4', 'Энтузиастов 8а', 'Левобережная 1а', 'Мира 14', 'ЗЖБИиДК 1', 
                    'Строителей 18', 'Панфиловская 33', 'Мастеркова 4']
    }
)

In [ ]:
rzd_auto = rzd.merge(auto, how='outer')
rzd_auto_air = rzd_auto.merge(air, how='outer')

In [ ]:
rzd_auto_air_address = rzd_auto_air.merge(client_base, how='outer')

# Задание 3

In [ ]:
# В задаче сквозной аналитики вам предоставили данные по местоположению пользователей. 
# Т. е. для каждого user_id известна последовательность координат (широта/долгота), 
# когда они требовались приложению для полноценной работы. 
# Как бы вы добавили эти сведения в таблицу визитов и покупок? 
# Для составления ответа можно использовать вопросы:
# 1. У каждого пользователя известен набор координат. 
# А для связывания с визитом или фактом покупки скорее всего потребуется одно-два числа. Как их получить?
# 2. Наборы координат одного и того же пользователя могут быть значительно удалены друг от друга. 
# Как это отразится на вопросе расчетах пункта 1?
# 3. Какие дополнительные признаки можно получить из координат? 
# Ведь это просто числа, которые сами по себе мало что дают.

In [ ]:
1. Необходимо объеденить датафрейм с user_id и координатами с датафреймом из user_id, визитов и покупок путем метода 
pd.merge(DataFrame, how='outer')

2. Удаленность координат не повлияет на расчеты. Удаленность координат свидетельствует о том, что пользователь
    заходил на сайт/приложение в разных местах. 

3. Координаты позволяют установить местонахождение пользователя в определенный момент времени. 
    В связке с координатами оффлайн магазина можно будет выявлять тех пользователей, которые находятся поблизости,
    после чего можно направлять им рекламные предложения, основанные на его предпочтениях исходя из ранних покупок.
    Кроме того, можно установить эффективность инвестиций в маркетинговые кампании в определенных источниках визитов
    пользователей, так как могут возникать ситуации, когда пользователь переходит на сайт/приложение с определенного
    источника и не приобретает ничего онлайн, но впоследствии он посещает оффлайн магазин и приобретает там товар.
    Соответственно, можно будет по связке координаты-user_id выявлять таких пользователей, которые посещали сайт/приложение,
    но приобретали товар не онлайн, а в оффлайн магазине, что позволяет оценить реальную эффективность маркетинговых
    мероприятий на различных источниках визитов - т.к. мы будем включать в анализ эффективности не только онлайн
    транзакции, но и оффлайн транзакции.